Solução (Gauss):
 [[ 1.04326923 -0.03269231]
 [ 2.26923077  0.10769231]
 [-1.08173077  0.21730769]]
Resíduos (Gauss): [1.77635684e-15 1.11022302e-16]

Solução (LU):
 [[ 1.04326923 -0.03269231]
 [ 2.26923077  0.10769231]
 [-1.08173077  0.21730769]]
Resíduos (LU): [1.77635684e-15 1.11022302e-16]


In [4]:
def _encontrar_linha_pivo(matriz, coluna, linha_inicial):
    """Encontra a linha com maior valor absoluto na coluna a partir da linha inicial."""
    linha_pivo = linha_inicial
    for linha in range(linha_inicial, len(matriz)):
        if abs(matriz[linha][coluna]) > abs(matriz[linha_pivo][coluna]):
            linha_pivo = linha
    return linha_pivo


def fatoracao_gaussiana(matriz):
    """Realiza a fatoração por eliminação gaussiana com pivotamento parcial."""
    tamanho_matriz = len(matriz)
    matriz_trabalho = [linha.copy() for linha in matriz]
    trocas_realizadas = []

    for coluna_pivo in range(tamanho_matriz):
        linha_pivo = _encontrar_linha_pivo(matriz_trabalho, coluna_pivo, coluna_pivo)

        if linha_pivo != coluna_pivo:
            trocas_realizadas.append((coluna_pivo, linha_pivo))
            matriz_trabalho[coluna_pivo], matriz_trabalho[linha_pivo] = matriz_trabalho[linha_pivo], matriz_trabalho[coluna_pivo]

        if abs(matriz_trabalho[coluna_pivo][coluna_pivo]) < 1e-10:
            raise ValueError("Matriz singular")

        for linha in range(coluna_pivo + 1, tamanho_matriz):
            fator_eliminacao = matriz_trabalho[linha][coluna_pivo] / matriz_trabalho[coluna_pivo][coluna_pivo]
            matriz_trabalho[linha][coluna_pivo] = fator_eliminacao
            for coluna in range(coluna_pivo + 1, tamanho_matriz):
                matriz_trabalho[linha][coluna] -= fator_eliminacao * matriz_trabalho[coluna_pivo][coluna]

    return matriz_trabalho, trocas_realizadas


def resolver_gauss(matriz_fatorada, trocas, vetor_constantes):
    """Resolve o sistema linear usando a matriz fatorada da eliminação gaussiana."""
    tamanho_matriz = len(matriz_fatorada)
    vetor_trabalho = vetor_constantes.copy()

    for coluna_pivo, linha_trocada in trocas:
        vetor_trabalho[coluna_pivo], vetor_trabalho[linha_trocada] = vetor_trabalho[linha_trocada], vetor_trabalho[coluna_pivo]

    for coluna_pivo in range(tamanho_matriz):
        for linha in range(coluna_pivo + 1, tamanho_matriz):
            vetor_trabalho[linha] -= matriz_fatorada[linha][coluna_pivo] * vetor_trabalho[coluna_pivo]

    solucao = [0.0] * tamanho_matriz
    for linha in reversed(range(tamanho_matriz)):
        solucao[linha] = vetor_trabalho[linha]
        for coluna in range(linha + 1, tamanho_matriz):
            solucao[linha] -= matriz_fatorada[linha][coluna] * solucao[coluna]
        solucao[linha] /= matriz_fatorada[linha][linha]

    return solucao


def fatoracao_lu(matriz):
    """Fatora a matriz em LU com pivotamento parcial."""
    tamanho_matriz = len(matriz)
    matriz_u = [linha.copy() for linha in matriz]
    matriz_l = [[0.0] * tamanho_matriz for _ in range(tamanho_matriz)]
    for diagonal in range(tamanho_matriz):
        matriz_l[diagonal][diagonal] = 1.0
    vetor_permutacao = list(range(tamanho_matriz))

    for coluna_pivo in range(tamanho_matriz):
        linha_pivo = _encontrar_linha_pivo(matriz_u, coluna_pivo, coluna_pivo)

        if linha_pivo != coluna_pivo:
            matriz_u[coluna_pivo], matriz_u[linha_pivo] = matriz_u[linha_pivo], matriz_u[coluna_pivo]
            matriz_l[coluna_pivo], matriz_l[linha_pivo] = matriz_l[linha_pivo], matriz_l[coluna_pivo]
            vetor_permutacao[coluna_pivo], vetor_permutacao[linha_pivo] = vetor_permutacao[linha_pivo], vetor_permutacao[coluna_pivo]

        if abs(matriz_u[coluna_pivo][coluna_pivo]) < 1e-10:
            raise ValueError("Matriz singular")

        for linha in range(coluna_pivo + 1, tamanho_matriz):
            fator_eliminacao = matriz_u[linha][coluna_pivo] / matriz_u[coluna_pivo][coluna_pivo]
            matriz_l[linha][coluna_pivo] = fator_eliminacao
            for coluna in range(coluna_pivo, tamanho_matriz):
                matriz_u[linha][coluna] -= fator_eliminacao * matriz_u[coluna_pivo][coluna]

    return vetor_permutacao, matriz_l, matriz_u


def resolver_lu(vetor_permutacao, matriz_l, matriz_u, vetor_constantes):
    """Resolve o sistema linear usando a fatoração LU."""
    tamanho_matriz = len(matriz_l)
    vetor_permutado = [vetor_constantes[indice] for indice in vetor_permutacao]

    vetor_y = [0.0] * tamanho_matriz
    for linha in range(tamanho_matriz):
        vetor_y[linha] = vetor_permutado[linha] - sum(matriz_l[linha][coluna] * vetor_y[coluna] for coluna in range(linha))

    vetor_x = [0.0] * tamanho_matriz
    for linha in reversed(range(tamanho_matriz)):
        vetor_x[linha] = vetor_y[linha] - sum(matriz_u[linha][coluna] * vetor_x[coluna] for coluna in range(linha + 1, tamanho_matriz))
        vetor_x[linha] /= matriz_u[linha][linha]

    return vetor_x


def verificar_solucao(matriz_coeficientes, vetor_solucao, vetor_constantes, tolerancia=1e-8):
    """Verifica se a solução satisfaz o sistema com tolerância especificada."""
    for linha_coef, termo_independente in zip(matriz_coeficientes, vetor_constantes):
        calculado = sum(coef * sol for coef, sol in zip(linha_coef, vetor_solucao))
        if abs(calculado - termo_independente) > tolerancia:
            return False
    return True


def principal():
    matriz_coeficientes = [
        [2, 1, -1],
        [-3, -1, 2],
        [-2, 1, 2]
    ]
    vetores_constantes = [
        [8, -11, -3],   # Solução esperada: [2.0, 3.0, -1.0]
        [-1, 7, 4]      # Outra solução
    ]

    print("Eliminação Gaussiana:")
    try:
        matriz_fatorada, trocas = fatoracao_gaussiana(matriz_coeficientes)
        for indice, vetor in enumerate(vetores_constantes, 1):
            solucao = resolver_gauss(matriz_fatorada, trocas, vetor)
            valido = verificar_solucao(matriz_coeficientes, solucao, vetor)
            print(f"Vetor {indice}: Solução = {solucao} {'válida' if valido else 'inválida'}")
    except ValueError as erro:
        print(f"Erro na fatoração: {erro}")

    print("\nFatoração LU:")
    try:
        permutacao, matriz_l, matriz_u = fatoracao_lu(matriz_coeficientes)
        for indice, vetor in enumerate(vetores_constantes, 1):
            solucao = resolver_lu(permutacao, matriz_l, matriz_u, vetor)
            valido = verificar_solucao(matriz_coeficientes, solucao, vetor)
            print(f"Vetor {indice}: Solução = {solucao} {'válida' if valido else 'inválida'}")
    except ValueError as erro:
        print(f"Erro na fatoração: {erro}")


if __name__ == "__main__":
    principal()

Eliminação Gaussiana:
Vetor 1: Solução = [2.0, 3.0000000000000004, -0.9999999999999999] válida
Vetor 2: Solução = [13.000000000000009, -8.000000000000005, 19.00000000000001] válida

Fatoração LU:
Vetor 1: Solução = [2.000000000000002, 2.999999999999999, -0.9999999999999971] válida
Vetor 2: Solução = [13.000000000000005, -8.000000000000004, 19.000000000000007] válida
